In [9]:
from pathlib import Path

import numpy as np
from numba import njit, prange

import pandas as pd

In [10]:
df = pd.read_csv(Path('..', '..', 'data', 'iris_csv.csv'))

for c in df.columns[0:4]:
    df[c] = (df[c]-df[c].mean())/df[c].std()

df['synth1'] = df['petallength']*df['petalwidth']
df['synth2'] = df['sepallength']*df['petallength']
df['synth3'] = df['sepallength']*df['petalwidth']

for name in df['class'].unique():
    df[f'label-{name}'] = df['class'].map(lambda x: 1 if x == name else 0)

In [11]:
np.random.seed(0)

setosa_idxs = np.arange(0, 50)
versicolor_idxs = np.arange(50, 100)
virginica_idxs = np.arange(100, 150)

p = np.random.permutation(np.arange(50))

setosa_train_idxs = setosa_idxs[p[0:10]]
setosa_test_idxs = setosa_idxs[p[10:]]

versicolor_train_idxs = versicolor_idxs[p[0:10]]
versicolor_test_idxs = versicolor_idxs[p[10:]]

virginica_train_idxs = virginica_idxs[p[0:10]]
virginica_test_idxs = virginica_idxs[p[10:]]

feature_columns = ['sepallength', 'sepalwidth', 'petallength', 'petalwidth']
label_columns = ['label-Iris-setosa', 'label-Iris-versicolor', 'label-Iris-virginica']

xTrain = np.vstack([
    df.iloc[setosa_train_idxs][feature_columns],
    df.iloc[versicolor_train_idxs][feature_columns],
    df.iloc[virginica_train_idxs][feature_columns]
])

yTrain = np.vstack([
    df.iloc[setosa_train_idxs][label_columns],
    df.iloc[versicolor_train_idxs][label_columns],
    df.iloc[virginica_train_idxs][label_columns]
])

xTest = np.vstack([
    df.iloc[setosa_test_idxs][feature_columns],
    df.iloc[versicolor_test_idxs][feature_columns],
    df.iloc[virginica_test_idxs][feature_columns]
])

yTest = np.vstack([
    df.iloc[setosa_test_idxs][label_columns],
    df.iloc[versicolor_test_idxs][label_columns],
    df.iloc[virginica_test_idxs][label_columns]
])

In [12]:
@njit(fastmath=True)
def F(x: np.ndarray) -> np.ndarray:
    # return x
    return np.maximum(np.zeros(x.shape), x)
    # return np.clip(x,-1, 1)
    # return np.tanh(x)


@njit(fastmath=True)
def dF(x: np.ndarray) -> np.ndarray:
    # return np.ones(x.shape)
    return 1 * (x > 0)
    # return np.array([0 if xi <= -1 or xi >= 1 else 1 for xi in x])
    # return 1-np.square(np.tanh(x))


@njit(fastmath=True)
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1/(1+np.exp(-x))


@njit(fastmath=True)
def dSigmoid(x: np.ndarray) -> np.ndarray:
    y = 1/(1+np.exp(-x))
    return y*(1-y)


@njit(fastmath=True)
def softmax(x: np.ndarray) -> np.ndarray:
    y = np.exp(x)
    return y/np.sum(y)

In [13]:
@njit(fastmath=True)
def grads(xBatch: np.ndarray, yBatch: np.ndarray, Wh: np.ndarray, Wy: np.ndarray, Bh: np.ndarray, b: np.ndarray) -> tuple[np.ndarray]:
    dWh = np.zeros(Wh.shape)
    dWy = np.zeros(Wy.shape)

    dBh = np.zeros(Bh.shape)
    db = np.zeros(b.shape)

    u = np.zeros((Wh.shape[0], Wh.shape[2]))
    h = np.zeros(Wh.shape[2])
    
    for i in prange(xBatch.shape[0]):
        h *= 0

        for j in range(Wh.shape[0]):
            u[j] = xBatch[i] @ Wh[j] + Bh[j]
            h += F(u[j])

        y = softmax(h @ Wy + b)
        
        for j in range(Wh.shape[0]):
            dWh[j] += np.outer(xBatch[i], (y - yBatch[i]) @ Wy.T * dF(u[j]))
            dBh[j] += (y - yBatch[i]) @ Wy.T * dF(u[j])
            
        dWy += np.outer(h, y - yBatch[i])
        db += (y - yBatch[i])
    
    return (dWh, dWy, dBh, db)


class MultiPerceptron:
    def __init__(self, nIn: int, nH:int, nL:int, nOut: int) -> None:
        self.nIn = nIn
        self.nH = nH
        self.nL = nL
        self.nOut = nOut
        
        self.Wh: np.ndarray = np.random.uniform(-1, 1, (nL, nIn, nH))
        self.Wy: np.ndarray = np.random.uniform(-1, 1, (nH, nOut))
        
        self.Bh: np.ndarray = np.zeros((nL, nH))
        self.b: np.ndarray = np.zeros(nOut)

    def predict(self, x:np.ndarray) -> np.ndarray:
        h = np.zeros(self.nH)

        for i in range(self.nL):
            h += F(x @ self.Wh[i] + self.Bh[i])
                
        return softmax(h @ self.Wy + self.b)

    def train(self, xTrain: np.ndarray, yTrain: np.ndarray, lr, batch_size, max_iter) -> None:
        n = xTrain.shape[0]

        for j in range(max_iter):
            idxs = np.random.choice(a=np.arange(n), size=batch_size, replace=False)
            
            dWh, dWy, dBh, db = grads(xTrain[idxs], yTrain[idxs], self.Wh, self.Wy, self.Bh, self.b)

            self.Wh -= lr*dWh
            self.Wy -= lr*dWy

            self.Bh -= lr*dBh
            self.b -= lr*db
        
    def loss(self, x: np.ndarray, y: np.ndarray) -> float:
        Ypred = np.array([self.predict(xi) for xi in x])
        h = -np.sum(y*np.log(Ypred), axis=1)
        return 1/x.shape[0] * np.sum(h)

In [14]:
nIn = 4
nH = 4
nL = 4
nOut = 3

lr = 1e-2
batch_size = 30
max_iter = 5000

model = MultiPerceptron(nIn, nH, nL, nOut)

print('untrained loss: {0:.6f}'.format(model.loss(xTest, yTest)))

model.train(xTrain, yTrain, lr, batch_size, max_iter)

print('trained loss: {0:.6f}'.format(model.loss(xTest, yTest)))

TP_count = 0
for x, y in zip(xTest, yTest):
    yPred = model.predict(x)
    TP_count += 1 if np.argmax(y) == np.argmax(yPred) else 0

accuracy = TP_count / xTest.shape[0]
print(accuracy)

untrained loss: 2.189754
trained loss: 0.324228
0.95


In [15]:
for x, y in zip(xTest, yTest):
    yPred = model.predict(x)
    print('X' if np.argmax(y) != np.argmax(yPred) else ' ' ,y.round(2), yPred.round(2))

  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [1 0 0] [1. 0. 0.]
  [0 1 0] [0.   0.58 0.42]
  [0 1 0] [0.  0.6 0.4]
  [0 1 0] [0. 1. 0.]
X [0 1 0] [0.   0.33 0.66]
  [0 1 0] [0. 1. 0.]
  [0 1 0] [0.01 0.99 0.  ]
  [0 1 0] [0.